In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from spicy_snow.retrieval import retrieval_from_parameters

In [2]:
in_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
in_dir = Path('/Users/rdcrlzh1/Documents/spicy-analysis/data/SnowEx-Data')
# in_dir = Path('~/spicy-snow/SnowEx-Data-Hans/').expanduser().resolve()

dss = {fp.stem: xr.open_dataset(fp) for fp in in_dir.glob('*.nc')}

In [32]:
res = pd.DataFrame()
all_pixels = 0
all_dry_pixel = 0
for stem, ds in dss.items():
    if stem == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(ds.sel(time = ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])
    site_name = stem.replace('_', ' ').split('-')[0].replace('Little Cottonwood', 'LCC').replace('Frasier', 'Fraser')

    x, y = ds['snow_depth'].sel(time = im_date, method = 'nearest'), ds['lidar-sd']
    pxl_cnt = (~np.isnan(x)) & (~np.isnan(y))

    x, y = ds['snow_depth'].sel(time = im_date, method = 'nearest'), ds['lidar-sd']
    x = x.where(ds['wet_snow'].sel(time = im_date, method = 'nearest') == 0)
    dry_pxl_cnt = (~np.isnan(x)) & (~np.isnan(y))

    res.loc[site_name, 'valid_pixels'] = pxl_cnt.sum().data.ravel()[0]
    res.loc[site_name, 'valid_dry_pixels'] = dry_pxl_cnt.sum().data.ravel()[0]
res.loc['all', ['valid_pixels', 'valid_dry_pixels']] = res.sum(axis = 0).values

In [33]:
res

,valid_pixels,valid_dry_pixels
Mores 2021,3813.0,581.0
Fraser 2021,6787.0,211.0
Dry Creek 2020,3792.0,1001.0
Banner 2021,16692.0,2531.0
Cameron 2021,2378.0,170.0
Banner 2020,16415.0,2571.0
Mores 2020,3694.0,1530.0
LCC 2021,2827.0,1045.0
Fraser 2020,3847.0,184.0
all,60245.0,9824.0
